In [ ]:
!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 numpy==1.24.4 --extra-index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 115.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 104.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
  Attempting uninstall: torch
    Found existing installation: torch 2.6.0+cu124
    Uninstalling torch-2.6.0+cu124:
      Successfully unin

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import os
from google.colab import drive
import sys
import torchvision.datasets as datasets
import numpy as np
import pickle
from PIL import Image
from torchvision.datasets import CIFAR10,CIFAR100

In [ ]:
!pip install torch_pruning
!pip install torchinfo
!pip install thop

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 5.5 MB/s eta 0:00:00


In [ ]:
drive.mount('/content/drive')
workspace_path = "/content/drive/MyDrive/DeepLearningCW"
os.makedirs(workspace_path, exist_ok=True)
%cd {workspace_path}
print(f"Working inside: {workspace_path}")
%cd /content/drive/MyDrive/DeepLearningCW/Model-LEGO

sys.path.append(os.getcwd())
print("Current Working Directory:", os.getcwd())
print("Python Path:", sys.path)

Mounted at /content/drive
/content/drive/MyDrive/DeepLearningCW
Working inside: /content/drive/MyDrive/DeepLearningCW
/content/drive/MyDrive/DeepLearningCW/Model-LEGO
Current Working Directory: /content/drive/MyDrive/DeepLearningCW/Model-LEGO
Python Path: ['/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/DeepLearningCW/Model-LEGO']


In [ ]:
from google.colab import files
import zipfile
import os


zip_path = "/content/cifar10_images.zip"


extract_folder = "cifar10_images.zip".replace(".zip", "")
extract_path = f"/content/{extract_folder}"


with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(" Upload and extraction complete.")
print("Extracted to:", extract_path)

 Upload and extraction complete.
Extracted to: /content/cifar10_images


In [ ]:
from google.colab import files
import zipfile
import os


zip_path = "/content/cifar100_images.zip"


extract_folder = "cifar100_images.zip".replace(".zip", "")
extract_path = f"/content/{extract_folder}"


with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(" Upload and extraction complete.")
print("Extracted to:", extract_path)

 Upload and extraction complete.
Extracted to: /content/cifar100_images


In [ ]:
from google.colab import files
import zipfile
import os


zip_path = "/content/tiny_imagenet_images.zip"


extract_folder = "tiny_imagenet_images.zip".replace(".zip", "")
extract_path = f"/content/{extract_folder}"


with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(" Upload and extraction complete.")
print("Extracted to:", extract_path)

In [ ]:
import os
import shutil

def filter_first_n_classes(source_dir, target_dir, n_classes):
    """
    Copies the first N sorted class folders from source_dir to target_dir.

    Args:
        source_dir (str): Path to the full dataset (e.g., test split).
        target_dir (str): Where to save the filtered dataset.
        n_classes (int): Number of class folders to copy.
    """
    os.makedirs(target_dir, exist_ok=True)


    class_folders = sorted([d for d in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, d))])


    selected_classes = class_folders[:n_classes]

    for class_name in selected_classes:
        src_path = os.path.join(source_dir, class_name)
        dst_path = os.path.join(target_dir, class_name)

        shutil.copytree(src_path, dst_path)
        print(f"Copied: {class_name}")

In [ ]:
import os
import shutil
from glob import glob

def merge_datasets(cifar10_path, cifar100_path, save_path):
    os.makedirs(save_path, exist_ok=True)


    cifar10_classes = sorted(os.listdir(cifar10_path))
    cifar100_classes = sorted(os.listdir(cifar100_path))


    combined_classes = {}
    for idx, class_name in enumerate(cifar10_classes):
        combined_classes[class_name] = idx
    for idx, class_name in enumerate(cifar100_classes):
        combined_classes[class_name] = idx + len(cifar10_classes)


    def copy_dataset(source_path, offset=0):
        class_names = sorted(os.listdir(source_path))
        for i, class_name in enumerate(class_names):
            new_label = i + offset
            class_dir = os.path.join(source_path, class_name)
            target_dir = os.path.join(save_path, f"{new_label:03d}_{class_name}")
            os.makedirs(target_dir, exist_ok=True)

            for image_file in glob(os.path.join(class_dir, "*")):
                shutil.copy(image_file, target_dir)

    print("Copying CIFAR-10...")
    copy_dataset(cifar10_path, offset=0)

    print("Copying CIFAR-100...")
    copy_dataset(cifar100_path, offset=len(cifar10_classes))

    print("Merge Complete! Combined dataset saved at:", save_path)

# Model Disassembly

## VGG16 + CIFAR10

In [ ]:
!python -m core.sample_select \
  --model_name 'vgg16' \
  --data_name 'cifar10' \
  --num_classes 10 \
  --model_path "/content/drive/MyDrive/DeepLearningCW/Model-LEGO/saved_models/VGG16_CIFAR10_seed44.pth"  \
  --data_dir  "/content/cifar10_images/train"\
  --save_dir "/content/samples_cifar10" \
  --num_samples 50

--------------------------------------------------
TRAIN ON: cuda
MODEL PATH: /content/drive/MyDrive/DeepLearningCW/Model-LEGO/saved_models/VGG16_CIFAR10_seed44.pth
DATA PATH: /content/cifar10_images/train
RESULT PATH: /content/samples_cifar10
--------------------------------------------------
--------------------------------------------------
LOAD MODEL: vgg16
NUM CLASSES: 10
--------------------------------------------------
--------------------------------------------------
DATA PATH: /content/cifar10_images/train
DATA NAME: cifar10 	|	DATA TYPE: test
--------------------------------------------------
100% 196/196 [00:07<00:00, 25.21it/s]
tensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1

In [ ]:
!python -m core.relevant_feature_identifying \
  --model_name 'vgg16' \
  --data_name cifar10 \
  --num_classes 10 \
  --model_path  "./saved_models/VGG16_CIFAR10_seed44.pth"\
  --data_dir "/content/samples_cifar10" \
  --save_dir "/content/"

--------------------------------------------------
TRAIN ON: cuda
DATA DIR: /content/samples_cifar10
SAVE DIR: /content/
--------------------------------------------------
--------------------------------------------------
LOAD MODEL: vgg16
NUM CLASSES: 10
--------------------------------------------------
--------------------------------------------------
DATA PATH: /content/samples_cifar10
DATA NAME: cifar10 	|	DATA TYPE: test
--------------------------------------------------
--------------------------------------------------
Model Layers: None
Model Modules: [Linear(in_features=4096, out_features=10, bias=True), Linear(in_features=4096, out_features=4096, bias=True), Linear(in_features=512, out_features=4096, bias=True), Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), paddin

In [ ]:
!ls core/


__init__.py			     model_disassemble.py	    relevant_feature_identifying.py
model_assemble.py		     parameter_scaling_modified.py  sample_select.py
model_decision_route_visualizing.py  parameter_scaling.py
model_disassemble_modified.py	     __pycache__


In [ ]:
!python -m core.model_disassemble_modified \
  --model_name 'vgg16' \
  --num_classes 10 \
  --model_path './saved_models/VGG16_CIFAR10_seed44.pth' \
  --save_dir '/content/saved_disassembled_models' \
  --mask_dir '/content/masks_c10/' \
  --disa_layers -1 \
  --disa_labels 0 1 2 \
  --model_save_name 'vgg16_cifar10_disassembled_seed44.pth'

--------------------------------------------------
SAVE DIR: /content/saved_disassembled_models
--------------------------------------------------
--------------------------------------------------
LOAD MODEL: vgg16
NUM CLASSES: 10
--------------------------------------------------
--------------------------------------------------
Model Layers: None
Model Modules: [Linear(in_features=4096, out_features=10, bias=True), Linear(in_features=4096, out_features=4096, bias=True), Linear(in_features=512, out_features=4096, bias=True), Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), 

In [ ]:
source_test_dir = '/content/cifar10_images/test'
target_test_dir = '/content/cifar10_subset/test'
num_classes_to_copy = 3

filter_first_n_classes(source_test_dir, target_test_dir, num_classes_to_copy)

Copied: airplane
Copied: automobile
Copied: bird


In [ ]:
!python -m engines.test \
  --model_name 'vgg16' \
  --data_name cifar10 \
  --num_classes 10 \
  --model_path "/content/saved_disassembled_models/vgg16_cifar10_disassembled_seed44.pth" \
  --data_dir "/content/cifar10_subset/test"

--------------------------------------------------
TEST ON: cuda
MODEL PATH: /content/saved_disassembled_models/vgg16_cifar10_disassembled_seed44.pth
DATA PATH: /content/cifar10_subset/test
--------------------------------------------------
--------------------------------------------------
DATA PATH: /content/cifar10_subset/test
DATA NAME: cifar10 	|	DATA TYPE: test
--------------------------------------------------
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
FLOPs = 0.21530303G
Params =

In [ ]:
!python -m engines.test \
  --model_name 'vgg16' \
  --data_name cifar10 \
  --num_classes 10 \
  --model_path "./saved_models/VGG16_CIFAR10_seed44.pth" \
  --data_dir "/content/cifar10_subset/test"

--------------------------------------------------
TEST ON: cuda
MODEL PATH: ./saved_models/VGG16_CIFAR10_seed44.pth
DATA PATH: /content/cifar10_subset/test
--------------------------------------------------
--------------------------------------------------
LOAD MODEL: vgg16
NUM CLASSES: 10
--------------------------------------------------
--------------------------------------------------
DATA PATH: /content/cifar10_subset/test
DATA NAME: cifar10 	|	DATA TYPE: test
--------------------------------------------------
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.


## VGG16 + CIFAR100

In [ ]:
!python -m engines.test \
  --model_name 'vgg16' \
  --data_name cifar100 \
  --num_classes 100 \
  --model_path "./saved_models/VGG16_CIFAR100_seed44.pth" \
  --data_dir "/content/cifar100_subset/test"

--------------------------------------------------
TEST ON: cuda
MODEL PATH: ./saved_models/VGG16_CIFAR100_seed44.pth
DATA PATH: /content/cifar100_subset/test
--------------------------------------------------
--------------------------------------------------
LOAD MODEL: vgg16
NUM CLASSES: 100
--------------------------------------------------
--------------------------------------------------
DATA PATH: /content/cifar100_subset/test
DATA NAME: cifar100 	|	DATA TYPE: test
--------------------------------------------------
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linea

In [ ]:
!python -m core.sample_select \
  --model_name 'vgg16' \
  --data_name 'cifar100' \
  --num_classes 100 \
  --model_path "./saved_models/VGG16_CIFAR100_seed44.pth"  \
  --data_dir  "/content/cifar100_images/train"\
  --save_dir "/content/samples_cifar100" \
  --num_samples 5

--------------------------------------------------
TRAIN ON: cuda
MODEL PATH: ./saved_models/VGG16_CIFAR100_seed44.pth
DATA PATH: /content/cifar100_images/train
RESULT PATH: /content/samples_cifar100
--------------------------------------------------
--------------------------------------------------
LOAD MODEL: vgg16
NUM CLASSES: 100
--------------------------------------------------
--------------------------------------------------
DATA PATH: /content/cifar100_images/train
DATA NAME: cifar100 	|	DATA TYPE: test
--------------------------------------------------
100% 196/196 [00:07<00:00, 24.96it/s]
tensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 

In [ ]:
!python -m core.relevant_feature_identifying \
  --model_name 'vgg16' \
  --data_name cifar100 \
  --num_classes 100 \
  --model_path  "./saved_models/VGG16_CIFAR100_seed44.pth"\
  --data_dir "/content/samples_cifar100" \
  --save_dir "/content/"

--------------------------------------------------
TRAIN ON: cuda
DATA DIR: /content/samples_cifar100
SAVE DIR: /content/
--------------------------------------------------
--------------------------------------------------
LOAD MODEL: vgg16
NUM CLASSES: 100
--------------------------------------------------
--------------------------------------------------
DATA PATH: /content/samples_cifar100
DATA NAME: cifar100 	|	DATA TYPE: test
--------------------------------------------------
--------------------------------------------------
Model Layers: None
Model Modules: [Linear(in_features=4096, out_features=100, bias=True), Linear(in_features=4096, out_features=4096, bias=True), Linear(in_features=512, out_features=4096, bias=True), Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), p

In [ ]:
!python -m core.model_disassemble_modified \
  --model_name 'vgg16' \
  --num_classes 100 \
  --model_path './saved_models/VGG16_CIFAR100_seed44.pth' \
  --save_dir '/content/saved_disassembled_models' \
  --mask_dir '/content/masks_c100/' \
  --disa_layers -1 \
  --disa_labels 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19  \
  --model_save_name 'vgg16_cifar100_disassembled_seed44.pth'


--------------------------------------------------
SAVE DIR: /content/saved_disassembled_models
--------------------------------------------------
--------------------------------------------------
LOAD MODEL: vgg16
NUM CLASSES: 100
--------------------------------------------------
--------------------------------------------------
Model Layers: None
Model Modules: [Linear(in_features=4096, out_features=100, bias=True), Linear(in_features=4096, out_features=4096, bias=True), Linear(in_features=512, out_features=4096, bias=True), Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1)

In [ ]:
source_test_dir = '/content/cifar100_images/test'
target_test_dir = '/content/cifar100_subset/test'
num_classes_to_copy = 20

filter_first_n_classes(source_test_dir, target_test_dir, num_classes_to_copy)

Copied: apple
Copied: aquarium_fish
Copied: baby
Copied: bear
Copied: beaver
Copied: bed
Copied: bee
Copied: beetle
Copied: bicycle
Copied: bottle
Copied: bowl
Copied: boy
Copied: bridge
Copied: bus
Copied: butterfly
Copied: camel
Copied: can
Copied: castle
Copied: caterpillar
Copied: cattle


In [ ]:
!python -m engines.test \
  --model_name 'vgg16' \
  --data_name cifar100 \
  --num_classes 100 \
  --model_path "/content/saved_disassembled_models/vgg16_cifar100_disassembled_seed44.pth" \
  --data_dir "/content/cifar100_subset/test"

--------------------------------------------------
TEST ON: cuda
MODEL PATH: /content/saved_disassembled_models/vgg16_cifar100_disassembled_seed44.pth
DATA PATH: /content/cifar100_subset/test
--------------------------------------------------
--------------------------------------------------
DATA PATH: /content/cifar100_subset/test
DATA NAME: cifar100 	|	DATA TYPE: test
--------------------------------------------------
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
FLOPs = 0.293081184G
Par

# Model Assembly

In [ ]:
source_test_dir = '/content/cifar10_images/train'
target_test_dir = '/content/cifar10_subset/train'
num_classes_to_copy = 3

filter_first_n_classes(source_test_dir, target_test_dir, num_classes_to_copy)

Copied: airplane
Copied: automobile
Copied: bird


In [ ]:
!python -m core.parameter_scaling \
  --model_name 'vgg16' \
  --data_name 'cifar10' \
  --num_classes 3 \
  --model_path "/content/saved_disassembled_models/vgg16_cifar10_disassembled_seed44.pth" \
  --data_dir "/content/cifar10_subset/train"

----------------------------------------------------------------------------------------------------
SCALE ON: cuda
MODEL PATH: /content/saved_disassembled_models/vgg16_cifar10_disassembled_seed44.pth
DATA DIR: /content/cifar10_subset/train
--------------------------------------------------
DATA PATH: /content/cifar10_subset/train
DATA NAME: cifar10 	|	DATA TYPE: test
--------------------------------------------------
100% 59/59 [00:02<00:00, 20.00it/s]
AVG SCORE RATIO:  tensor([1.8072, 1.5766, 1.5485], dtype=torch.float64)
Reciprocal AVG SCORE RATIO:  tensor([0.5533, 0.6343, 0.6458], dtype=torch.float64)
PICTURE NUM:  tensor([4989, 4971, 4902])
--------------------------------------------------
Model Layers: None
Model Modules: [Linear(in_features=326, out_features=3, bias=True), Linear(in_features=739, out_features=326, bias=True), Linear(in_features=270, out_features=739, bias=True), Conv2d(320, 270, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), Conv2d(299, 320, kernel_size=(3

In [ ]:
source_test_dir = '/content/cifar100_images/train'
target_test_dir = '/content/cifar100_subset/train'
num_classes_to_copy = 20

filter_first_n_classes(source_test_dir, target_test_dir, num_classes_to_copy)

Copied: apple
Copied: aquarium_fish
Copied: baby
Copied: bear
Copied: beaver
Copied: bed
Copied: bee
Copied: beetle
Copied: bicycle
Copied: bottle
Copied: bowl
Copied: boy
Copied: bridge
Copied: bus
Copied: butterfly
Copied: camel
Copied: can
Copied: castle
Copied: caterpillar
Copied: cattle


In [ ]:
!python -m core.parameter_scaling \
  --model_name 'vgg16' \
  --data_name 'cifar100' \
  --num_classes 20 \
  --model_path "/content/saved_disassembled_models/vgg16_cifar100_disassembled_seed44.pth" \
  --data_dir "/content/cifar100_subset/train"

----------------------------------------------------------------------------------------------------
SCALE ON: cuda
MODEL PATH: /content/saved_disassembled_models/vgg16_cifar100_disassembled_seed44.pth
DATA DIR: /content/cifar100_subset/train
--------------------------------------------------
DATA PATH: /content/cifar100_subset/train
DATA NAME: cifar100 	|	DATA TYPE: test
--------------------------------------------------
100% 40/40 [00:02<00:00, 19.11it/s]
AVG SCORE RATIO:  tensor([6.2066, 4.5461, 8.2861, 6.8675, 6.2799, 8.5650, 7.8145, 6.8785, 8.4079,
        8.5715, 7.0854, 7.0331, 6.8647, 6.3234, 8.4489, 9.0107, 8.3316, 8.7219,
        8.2383, 9.3942], dtype=torch.float64)
Reciprocal AVG SCORE RATIO:  tensor([0.1611, 0.2200, 0.1207, 0.1456, 0.1592, 0.1168, 0.1280, 0.1454, 0.1189,
        0.1167, 0.1411, 0.1422, 0.1457, 0.1581, 0.1184, 0.1110, 0.1200, 0.1147,
        0.1214, 0.1064], dtype=torch.float64)
PICTURE NUM:  tensor([484, 357, 500, 495, 495, 500, 500, 495, 500, 500, 497, 48

In [ ]:
!python -m core.model_assemble \
  --model1_path "/content/saved_disassembled_models/vgg16_cifar10_disassembled_seed44_scale.pth" \
  --model2_path "/content/saved_disassembled_models/vgg16_cifar100_disassembled_seed44_scale.pth" \
  --asse_path "/content/model_assembled_s.pth"

=================> Architecture Assembling
VGG(
  (features): Sequential(
    (0): Conv2d(3, 115, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(115, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(115, 124, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(124, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(124, 252, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(252, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(252, 255, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(255, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilat

In [ ]:
merge_datasets(
    cifar10_path='/content/cifar10_subset/test',
    cifar100_path='/content/cifar100_subset/test',
    save_path='/content/cifar_merged/test'
)

In [ ]:
!python -m engines.test \
  --model_name 'vgg16' \
  --data_name cifar10 \
  --num_classes 23 \
  --model_path "/content/model_assembled_s.pth" \
  --data_dir "/content/cifar_merged/test"

--------------------------------------------------
TEST ON: cuda
MODEL PATH: /content/model_assembled_s.pth
DATA PATH: /content/cifar_merged/test
--------------------------------------------------
--------------------------------------------------
DATA PATH: /content/cifar_merged/test
DATA NAME: cifar10 	|	DATA TYPE: test
--------------------------------------------------
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
FLOPs = 0.994255687G
Params = 42.705066M
Test[19/20]  Loss[2.4750e+00]  Ac